In [1]:
%cd ..

/home/jbananafish/Desktop/Master/Thesis/code/gcnboost


In [2]:
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, Linear, GraphConv, GATConv, to_hetero
import torch_geometric.transforms as T
import torch_geometric.nn as operators
from sklearn.metrics import confusion_matrix as confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score
import numpy as np

from src.data.artgraph import ArtGraph

In [3]:
torch.manual_seed(1)
torch.cuda.manual_seed(1)

In [4]:
#base_data = ArtGraph("./ekg", preprocess='node2vec', features=True, type='ekg')
base_data = ArtGraph("data", preprocess='constant', features=True, type='ekg')

Extracting data/artgraph/raw/artgraph.zip
Processing...
Done!


In [5]:
data = base_data[0]

In [6]:
data

HeteroData(
  artwork={
    x=[61477, 128],
    y_artist=[61477],
    y_style=[61477],
    y_genre=[61477],
    train_mask=[61477],
    val_mask=[61477],
    test_mask=[61477]
  },
  artist={ x=[300, 128] },
  gallery={ x=[1090, 128] },
  city={ x=[665, 128] },
  country={ x=[64, 128] },
  style={ x=[83, 128] },
  period={ x=[53, 128] },
  genre={ x=[50, 128] },
  serie={ x=[610, 128] },
  auction={ x=[5, 128] },
  tag={ x=[5146, 128] },
  media={ x=[160, 128] },
  subject={ x=[2161, 128] },
  training_node={ x=[108, 128] },
  field={ x=[65, 128] },
  movement={ x=[121, 128] },
  people={ x=[48, 128] },
  (artist, influenced_rel, artist)={ edge_index=[2, 31] },
  (artist, subject_rel, subject)={ edge_index=[2, 3648] },
  (artist, training_rel, training_node)={ edge_index=[2, 130] },
  (artist, field_rel, field)={ edge_index=[2, 323] },
  (artist, movement_rel, movement)={ edge_index=[2, 286] },
  (artist, patrons_rel, people)={ edge_index=[2, 45] },
  (artist, teacher_rel, artist)={ ed

In [7]:
data['artist']

{'x': tensor([[ 0.6379, -0.2165,  0.6949,  ...,  0.1997,  0.4008,  0.3378],
        [ 0.0807, -0.0510, -0.1475,  ...,  0.1655, -0.4353,  0.1827],
        [ 0.3808,  0.6442,  1.3926,  ...,  0.2606,  0.1476, -0.4818],
        ...,
        [ 0.4290, -0.1619,  0.3100,  ..., -0.2008, -0.0391,  0.4392],
        [ 0.0748,  0.1289, -0.1568,  ..., -0.3241, -0.3156,  0.1110],
        [ 0.1364, -0.2297,  1.2567,  ..., -0.3991,  0.1389,  0.0585]])}

In [55]:
del data['gallery']
del data['city']
del data['country']
del data['auction']
del data['media']
del data['tag']
del data['period']
del data['serie']
del data['subject']
del data['field']
del data['movement']
del data['people']
del data['training_node']

del data['artwork', 'auction_rel', 'auction']
del data['city', 'country_rel', 'country']
del data['gallery', 'city_rel', 'city']
del data['gallery', 'country_rel', 'country']
del data['artwork', 'locatedin_rel', 'gallery']
del data['artwork', 'completedin_rel', 'city']

del data['artwork', 'media_rel', 'media']
del data['artwork', 'about_rel', 'tag']
del data['artwork', 'period_rel', 'period']
del data['artwork', 'serie_rel', 'serie']
del data['people', 'patrons_rel', 'artist']

del data['artist', 'influenced_rel', 'artist']
del data['artist', 'subject_rel', 'subject']
del data['artist', 'training_rel', 'artist']
del data['artist', 'training_rel', 'training_node']
del data['artist', 'field_rel', 'field']
del data['artist', 'movement_rel', 'movement']
del data['artist', 'patrons_rel', 'people']
del data['artist', 'teacher_rel', 'artist']

In [56]:
data

HeteroData(
  artwork={
    x=[61477, 128],
    y_artist=[61477],
    y_style=[61477],
    y_genre=[61477],
    train_mask=[61477],
    val_mask=[61477],
    test_mask=[61477]
  },
  artist={ x=[300, 128] },
  style={ x=[83, 128] },
  genre={ x=[50, 128] },
  (artwork, genre_rel, genre)={ edge_index=[2, 61820] },
  (artwork, style_rel, style)={ edge_index=[2, 63157] },
  (artwork, author_rel, artist)={ edge_index=[2, 61477] }
)

In [42]:
data = T.ToUndirected()(data)

In [36]:
data

HeteroData(
  artwork={
    x=[61477, 128],
    y_artist=[61477],
    y_style=[61477],
    y_genre=[61477],
    train_mask=[61477],
    val_mask=[61477],
    test_mask=[61477]
  },
  artist={ num_nodes=300 },
  gallery={ num_nodes=1090 },
  city={ num_nodes=665 },
  country={ num_nodes=64 },
  style={ num_nodes=83 },
  period={ num_nodes=53 },
  genre={ num_nodes=50 },
  serie={ num_nodes=610 },
  auction={ num_nodes=5 },
  tag={ num_nodes=5146 },
  media={ num_nodes=160 },
  subject={ num_nodes=2161 },
  training_node={ num_nodes=108 },
  field={ num_nodes=65 },
  movement={ num_nodes=121 },
  people={ num_nodes=48 },
  (artist, influenced_rel, artist)={ edge_index=[2, 62] },
  (artist, subject_rel, subject)={ edge_index=[2, 3648] },
  (artist, training_rel, training_node)={ edge_index=[2, 130] },
  (artist, field_rel, field)={ edge_index=[2, 323] },
  (artist, movement_rel, movement)={ edge_index=[2, 286] },
  (artist, patrons_rel, people)={ edge_index=[2, 45] },
  (artist, teacher_r

In [43]:
data = data.to_homogeneous()

In [44]:
data

Data(x=[72206, 128], edge_index=[2, 947934], edge_type=[947934])

In [44]:
#data = T.TwoHop()(data)

## Some graph-level information

In [60]:
print(f"Number of artist classes: {base_data.num_classes['artist']}")
print(f"Number of style classes: {base_data.num_classes['style']}")
print(f"Number of genre classes: {base_data.num_classes['genre']}")
print(f"Number of input features: {base_data.num_features}")

Number of artist classes: 300
Number of style classes: 83
Number of genre classes: 50
Number of input features: 128


## Some node-level information

In [13]:
data = base_data[0]
base_data

ArtGraph()

In [14]:
print(data)

HeteroData(
  artwork={
    num_nodes=61477,
    y_artist=[61477],
    y_style=[61477],
    y_genre=[61477],
    train_mask=[61477],
    val_mask=[61477],
    test_mask=[61477]
  },
  artist={ num_nodes=300 },
  gallery={ num_nodes=1090 },
  city={ num_nodes=665 },
  country={ num_nodes=64 },
  style={ num_nodes=83 },
  period={ num_nodes=53 },
  genre={ num_nodes=50 },
  serie={ num_nodes=610 },
  auction={ num_nodes=5 },
  tag={ num_nodes=5146 },
  media={ num_nodes=160 },
  subject={ num_nodes=2161 },
  training_node={ num_nodes=108 },
  field={ num_nodes=65 },
  movement={ num_nodes=121 },
  people={ num_nodes=48 },
  (artist, influenced_rel, artist)={ edge_index=[2, 62] },
  (artist, subject_rel, subject)={ edge_index=[2, 3648] },
  (artist, training_rel, training_node)={ edge_index=[2, 130] },
  (artist, field_rel, field)={ edge_index=[2, 323] },
  (artist, movement_rel, movement)={ edge_index=[2, 286] },
  (artist, patrons_rel, people)={ edge_index=[2, 45] },
  (artist, teacher_

In [10]:
del data['gallery']
del data['city']
del data['country']
del data['auction']

del data['artwork', 'auction_rel', 'auction']
del data['auction', 'rev_auction_rel', 'artwork']

del data['city', 'country_rel', 'country']
del data['country', 'rev_country_rel', 'city']

del data['gallery', 'city_rel', 'city']
del data['city', 'rev_city_rel', 'gallery']

del data['gallery', 'country_rel', 'country']
del data['country', 'rev_country_rel', 'gallery']

del data['artwork', 'locatedin_rel', 'gallery']
del data['gallery', 'rev_locatedin_rel', 'artwork']

del data['artwork', 'completedin_rel', 'city']
del data['city', 'rev_completedin_rel', 'artwork']

In [11]:
data

HeteroData(
  artwork={
    x=[61477, 128],
    y_artist=[61477],
    y_style=[61477],
    y_genre=[61477],
    train_mask=[61477],
    val_mask=[61477],
    test_mask=[61477]
  },
  artist={ x=[300, 128] },
  style={ x=[83, 128] },
  period={ x=[53, 128] },
  genre={ x=[50, 128] },
  serie={ x=[610, 128] },
  tag={ x=[5146, 128] },
  media={ x=[160, 128] },
  subject={ x=[2161, 128] },
  training_node={ x=[108, 128] },
  field={ x=[65, 128] },
  movement={ x=[121, 128] },
  people={ x=[48, 128] },
  (artist, influenced_rel, artist)={ edge_index=[2, 62] },
  (artist, subject_rel, subject)={ edge_index=[2, 3648] },
  (artist, training_rel, training_node)={ edge_index=[2, 130] },
  (artist, field_rel, field)={ edge_index=[2, 323] },
  (artist, movement_rel, movement)={ edge_index=[2, 286] },
  (artist, patrons_rel, people)={ edge_index=[2, 45] },
  (artist, teacher_rel, artist)={ edge_index=[2, 12] },
  (artwork, media_rel, media)={ edge_index=[2, 56347] },
  (artwork, about_rel, tag)={ 

In [61]:
class HomoGNN(torch.nn.Module):
    def __init__(self, operator=GCNConv, activation=torch.nn.ReLU, input_channels=128, hidden_channels=16, out_channels=300, num_layers=1, dropout=0.5, bn=False, skip=False):
        super(HomoGNN, self).__init__()
        self.dropout = dropout
        self.skip = skip
        self.bn = bn
        self.activation = activation
        self.convs = torch.nn.ModuleList()
        self.lins = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        #self.convs.append(operator(input_channels, hidden_channels))
        for _ in range(num_layers):
            conv = operator(-1, hidden_channels)
            lin = Linear(-1, hidden_channels)
            bn = torch.nn.BatchNorm1d(hidden_channels)
            self.convs.append(conv)
            self.lins.append(lin)
            self.bns.append(bn)
        self.conv_out = operator(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            if self.skip:
                x = conv(x, edge_index) + self.lins[i](x)
            else:
                x = conv(x, edge_index)
            x = self.bns[i](x)
            x= self.activation(x)
            if self.training:
                x = F.dropout(x, self.dropout)
        x = self.conv_out(x, edge_index)
        return F.log_softmax(x, dim=1)

In [62]:
class HomoSGNN(torch.nn.Module):
    def __init__(self, operator, activation, input_channels, hidden_channels, out_channels, n_layers, dropout, bn, skip):
        super(HomoSGNN, self).__init__()
        self.gnn = HomoGNN(operator, activation, input_channels, hidden_channels, out_channels, n_layers, dropout, bn, skip)

    def forward(self, x, edge_index):
        return [self.gnn(x, edge_index)]

class HomoMGNN(torch.nn.Module):
    def __init__(self, operator, activation, input_channels, hidden_channels, out_channels, n_layers, dropout, bn, skip):
        super(HomoMGNN, self).__init__()
        self.gnn_artist = HomoGNN(operator, activation, input_channels, hidden_channels, out_channels['artist'], n_layers, dropout, bn, skip)
        self.gnn_style = HomoGNN(operator, activation, input_channels, hidden_channels, out_channels['style'], n_layers, dropout, bn, skip)
        self.gnn_genre = HomoGNN(operator, activation, input_channels, hidden_channels, out_channels['genre'], n_layers, dropout, bn, skip)

    def forward(self, x, edge_index):
        return [self.gnn_artist(x, edge_index), self.gnn_style(x, edge_index), self.gnn_genre(x, edge_index)]

In [177]:
class ArtGraphGCNBoost:
    operator_registry = {
        'SAGEConv': operators.SAGEConv,
        'GraphConv': operators.GraphConv,
        'GATConv': operators.GATConv,
        'GCNConv': operators.GCNConv
    }

    activation_registry = {
        'relu': torch.nn.ReLU(),
        'prelu': torch.nn.PReLU()
    }

    map_id2labels = {
        0: 'artist',
        1: 'style',
        2: 'genre'
    }

    map_labels2id = {
        'artist': 0,
        'style': 1,
        'genre': 2
    }

    def __init__(self, args, training_mode='multi_task'):
        
        self.traning_mode = training_mode
        assert training_mode in ['multi_task', 'single_task']
        assert args.operator in self.operator_registry.keys()

        self.base_data, self.data, self.y, self.model, self.optimizer = self._bootstrap(args)
        self.artworks = self.base_data[0]['artwork']
        self.train_mask = self.artworks.train_mask
        self.val_mask = self.artworks.val_mask
        self.test_mask = self.artworks.test_mask

    def _bootstrap(self, args):
        base_data = ArtGraph("data", preprocess='node2vec', transform=T.ToUndirected(), features=True, type='ekg')
        data = base_data[0]
        data = data.to_homogeneous()
        if self.traning_mode == 'multi_task':
            model = HomoMGNN(operator=self.operator_registry[args.operator],
                                activation=self.activation_registry[args.activation],
                                input_channels=base_data.num_features,
                                hidden_channels=args.hidden,
                                out_channels=base_data.num_classes,
                                n_layers=args.nlayers,
                                dropout=args.dropout,
                                bn=args.bn,
                                skip=args.skip)
            y = torch.stack([base_data[0]['artwork'].y_artist, base_data[0]['artwork'].y_style, base_data[0]['artwork'].y_genre])
        if self.traning_mode == 'single_task':
            model = HomoSGNN(operator=self.operator_registry[args.operator],
                                activation=self.activation_registry[args.activation],
                                input_channels=base_data.num_features,
                                hidden_channels=args.hidden,
                                out_channels=base_data.num_classes[args.label],
                                n_layers=args.nlayers,
                                dropout=args.dropout,
                                bn=args.bn,
                                skip=args.skip)
            y = torch.stack([base_data[0]['artwork'][f'y_{args.label}']])
        
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=3e-4)
        
        return base_data, data, y, model, optimizer
    
    def get_accuracy(self, predicted, labels):
        return accuracy_score(labels, predicted.argmax(dim=1))

    def get_accuracies_homo(self, predicted, labels, mask):
        size = self.train_mask.shape[0]
        accuracies = [] 
        for i, _ in enumerate(labels):
            accuracies.append(self.get_accuracy(predicted[i][:size][mask], labels[i][mask]))
        return accuracies

    def get_loss(self, predicted, labels):
        return F.nll_loss(predicted, labels.type(torch.LongTensor))
    
    def get_losses_homo(self, predicted, labels, mask):
        size = self.train_mask.shape[0]
        losses = []
        for i, _ in enumerate(labels):
            losses.append(self.get_loss(predicted[i][:size][mask], labels[i][mask]))
        return losses

    def get_precision_recall_homo(self, predicted, labels):
        size = self.train_mask.shape[0]
        cms = []
        ps = []
        rs = []
        for i, _ in enumerate(labels):
            y_true = labels[i][self.test_mask].numpy()
            y_pred = predicted[i][:size][self.test_mask].argmax(dim=1).numpy()
            cm = confusion_matrix(y_true, y_pred)
            cms.append(cm)
            ps.append(precision_score(y_true, y_pred, average='macro', zero_division=0))
            rs.append(recall_score(y_true, y_pred, average='macro', zero_division=0))
        
        return cms, ps, rs

    def homo_training(self):
        self.model.train()
        self.optimizer.zero_grad()
        out = self.model(self.data.x, self.data.edge_index)

        train_losses = self.get_losses_homo(out, self.y, self.train_mask)
        train_total_loss = sum(train_losses)

        train_total_loss.backward()
        self.optimizer.step()

        train_accuracies = self.get_accuracies_homo(out, self.y, self.train_mask)

        return out, train_losses, train_accuracies

    @torch.no_grad()
    def homo_test(self):
        self.model.eval()
        out = self.model(self.data.x, self.data.edge_index)
        val_losses = self.get_losses_homo(out, self.y, self.val_mask)
        test_losses = self.get_losses_homo(out, self.y, self.test_mask)

        val_accuracies = self.get_accuracies_homo(out, self.y, self.val_mask)
        test_accuracies = self.get_accuracies_homo(out, self.y, self.test_mask)

        cms, ps, rs = self.get_precision_recall_homo(out, self.y)

        return val_losses, val_accuracies, test_losses, test_accuracies, cms, ps, rs

In [178]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--exp', type=str, default='default', help='Experiment name.')
parser.add_argument('--type', type=str, default='homo', help='Graph type (hetero|homo).')
parser.add_argument('--mode', type=str, default='single_task', help='Training mode (multi_task|single_task).')
parser.add_argument('--label', type=str, default='genre', help='Label to predict (artist|style|genre).')
parser.add_argument('--epochs', type=int, default=1, help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate.')
parser.add_argument('--hidden', type=int, default=16, help='Number of hidden units.')
parser.add_argument('--nlayers', type=int, default=1, help='Number of layers.')
parser.add_argument('--dropout', type=float, default=0, help='Dropout rate (1 - keep probability).')
parser.add_argument('--operator', type=str, default='GCNConv', help='The graph convolutional operator.')
parser.add_argument('--activation', type=str, default='prelu', help='The activation function.')
parser.add_argument('--aggr', type=str, default='sum', help='Aggregation function.')
parser.add_argument('--skip', action='store_true', default='False', help='Add skip connection.')
parser.add_argument('--bn', action='store_true', default='True', help='Add batch normalization.')
args, unknown = parser.parse_known_args()

In [179]:
gcn = ArtGraphGCNBoost(args, training_mode=args.mode)

In [180]:
for epoch in tqdm(range(0, args.epochs)):
    out, train_losses, train_accuracies = gcn.homo_training()
    val_losses, val_accuracies, test_losses, test_accuracies, cms, ps, rs = gcn.homo_test()
    if epoch % 25 ==0:
        if args.mode == 'multi_task':
            for i, train_loss_acc in enumerate(zip(train_losses, train_accuracies)):
                print(f'{gcn.map_id2labels[i]}_train_loss', round(train_loss_acc[0].detach().item(), 4))
                print(f'{gcn.map_id2labels[i]}_train_accuracy', round(train_loss_acc[1].item(), 2) * 100)
            for i, val_loss_acc in enumerate(zip(val_losses, val_accuracies)):
                print(f'{gcn.map_id2labels[i]}_val_loss', round(val_loss_acc[0].detach().item(), 4))
                print(f'{gcn.map_id2labels[i]}_val_accuracy', round(val_loss_acc[1].item(), 2) * 100)
            for i, test_loss_acc_prec_rec in enumerate(zip(test_losses, test_accuracies, ps, rs)):
                print(f'{gcn.map_id2labels[i]}_test_loss', round(test_loss_acc_prec_rec[0].detach().item(), 4))
                print(f'{gcn.map_id2labels[i]}_test_accuracy', round(test_loss_acc_prec_rec[1].item(), 2) * 100)
                print(f'{gcn.map_id2labels[i]}_precision', test_loss_acc_prec_rec[2] * 100)
                print(f'{gcn.map_id2labels[i]}_recall', test_loss_acc_prec_rec[3] * 100)
        if args.mode == 'single_task':
            print(f'{args.label}_train_loss', round(train_losses[0].detach().item(), 4))
            print(f'{args.label}_train_accuracy', round(train_accuracies[0].item(), 2) * 100)
            print(f'{args.label}_val_loss', round(val_losses[0].detach().item(), 4))
            print(f'{args.label}_val_accuracy', round(val_accuracies[0].item(), 2) * 100)
            print(f'{args.label}_test_loss', round(test_losses[0].detach().item(), 4))
            print(f'{args.label}_test_accuracy', round(test_accuracies[0].item(), 2) * 100)
            print(f'{args.label}_test_precision', ps[0] * 100)
            print(f'{args.label}_test_recall', rs[0] * 100)

100%|██████████| 1/1 [00:03<00:00,  3.40s/it]

genre_train_loss 3.9765
genre_train_accuracy 1.0
genre_val_loss 3.9438
genre_val_accuracy 3.0
genre_test_loss 3.9491
genre_test_accuracy 3.0
genre_test_precision 0.7665308989110079
genre_test_recall 0.9954619914462736
